In [1]:
import json

In [2]:
def load_documents(file_path):
    """Load documents from a JSON file"""
    with open(file_path, 'r', encoding='utf-8') as file:
            documents = json.load(file)
    print(f"Successfully loaded {len(documents)} documents from {file_path}")
    return documents

In [3]:
documents = load_documents("final_updated.json")

Successfully loaded 862 documents from final_updated.json


In [4]:
# Prepare texts and metadata for insertion
texts = []
metadatas = []

for doc in documents:
    # Get translations safely
    t1 = doc.get("translation1", "")
    t2 = doc.get("translation2", "")
    t3 = doc.get("translation3", "")

    # Get other fields
    arabic = doc.get("arabic", "")
    tags = doc.get("tags", [])

    # Combine all relevant content into searchable text
    combined_text_parts = [arabic, t1, t2, t3] + tags
    text = " ".join(combined_text_parts).strip()
    texts.append(text)

    # Build metadata dictionary
    metadata = {
        "id": doc.get("id"),
        "tags": tags
    }
    metadatas.append(metadata)

In [5]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-m3")

C:\Users\MuhammadUsman\AppData\Local\Temp\ipykernel_2864\4281357771.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-m3")


In [6]:
from langchain_community.vectorstores import SQLiteVec

In [7]:
connection = SQLiteVec.create_connection(db_file="recommendation.db")

In [8]:
vector_store = SQLiteVec(
    table="recommendation", embedding=embedding_function, connection=connection
)

In [9]:
vector_store.add_texts(texts=texts, metadatas=metadatas)

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [10]:
# List of queries
queries = [
    "What is the waiting period ('iddah) for a divorced woman?",
   "Are relatives eligible to receive zakat?",
   "How should inheritance be distributed among siblings in Islam?",
   "Is zakat due on gold jewelry that is not worn regularly?",
   "What are the Islamic rulings on adoption, and does the child inherit from adoptive parents?",
   "Can a Muslim woman marry a non-Muslim man who is from Ahl al-Kitab?",
   "What should a person do if they miss a fast during Ramadan due to illness or travel?",
   "I've divorced my wife in anger but now we both want to reconcile. Is there anything in the Qur'an about taking her back?",
   "I've made a lot of mistakes in life and now I want to return to Islam.",
   "I had to take a loan with interest due to financial hardship.",
]

In [11]:
# Batch process each query
for query in queries:
    print(f"Query: {query}")
    results = vector_store.similarity_search(query, k=3)
    for i, result in enumerate(results):
        print(f"  Result {i+1}:")
        if result.metadata:
            print(f"    ID: {result.metadata.get('id', 'N/A')}")
            print(f"    Tag: {result.metadata.get('tags', 'N/A')}")
            print("")
    print("=" * 100)

Query: What is the waiting period ('iddah) for a divorced woman?
  Result 1:
    ID: 2|228
    Tag: ['rights of divorced women', 'Islam on reconciliation in divorce']

  Result 2:
    ID: 33|49
    Tag: ['Quran on marriage and divorce', 'guidelines for Iddah and gifts in divorce']

  Result 3:
    ID: 2|234
    Tag: ["Quran on widow's waiting period", 'Islamic guidance for widows']

Query: Are relatives eligible to receive zakat?
  Result 1:
    ID: 4|8
    Tag: ['Quran on inheritance justice', 'support for orphans and the poor']

  Result 2:
    ID: 9|11
    Tag: ['Quran on repentance and charity', 'brotherhood in faith']

  Result 3:
    ID: 2|110
    Tag: ['importance of prayer and zakat', 'Islamic charity and good deeds']

Query: How should inheritance be distributed among siblings in Islam?
  Result 1:
    ID: 4|176
    Tag: ['Quran on inheritance laws', 'gender equity in Islamic inheritance']

  Result 2:
    ID: 4|12
    Tag: ['inheritance laws in Islam', 'rights of wives in inh